<a href="https://colab.research.google.com/github/caballeroarturo/Ejemplo7/blob/main/Ejemplo_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplo 3. Variantes en la lectura de BDD con R

Ahora utilizaremos otra opción para realizar queries a una BDD con la ayuda
de dplyr que sustituye a SELECT en MySQL y el operador %>%, hay que recordar
que con este comando también podemos realizar búsquedas de forma local.

Comenzamos instalando las paqueterías necesarias y cargándolas a R

```
install.packages("pool")

install.packages("dbplyr")

```



In [30]:
install.packages("pacman")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [31]:
pacman::p_load(dbplyr,pool,RMySQL,dplyr,DBI)

Se realiza la lectura de la BDD con el comando dbPool, los demás parámetros
se siguen utilizando igual que el ejemplo anterior

In [6]:
my_db <- dbPool(
  RMySQL::MySQL(),
  dbname = "shinydemo",
  host = "shiny-demo.csa7qlmguqrf.us-east-1.rds.amazonaws.com",
  username = "guest",
  password = "guest"
)

Para ver el contenido de la BDD y realizar una búsqueda se procede de la
siguiente manera

In [7]:
dbListTables(my_db)

[1] "City"            "Country"         "CountryLanguage"

In [8]:
my_db

<Pool> of MySQLConnection objects
  Objects checked out: 0
  Available in pool: 1
  Max size: Inf
  Valid: TRUE

In [11]:
# Obtener los primeros 5 registros de Country

my_db %>% tbl("Country") %>% head(5) # library(dplyr)

# Source:   SQL [5 x 15]
# Database: mysql 5.5.5-10.6.16-MariaDB-log [@shiny-demo.csa7qlmguqrf.us-east-1.rds.amazonaws.com:/shinydemo]
  Code  Name    Continent Region SurfaceArea IndepYear Population LifeExpectancy
  <chr> <chr>   <chr>     <chr>        <dbl>     <int>      <int>          <dbl>
1 ABW   Aruba   North Am… Carib…         193        NA     103000           78.4
2 AFG   Afghan… Asia      South…      652090      1919   22720000           45.9
3 AGO   Angola  Africa    Centr…     1246700      1975   12878000           38.3
4 AIA   Anguil… North Am… Carib…          96        NA       8000           76.1
5 ALB   Albania Europe    South…       28748      1912    3401200           71.6
# ℹ 7 more variables: GNP <dbl>, GNPOld <dbl>, LocalName <chr>,
#   GovernmentForm <chr>, HeadOfState <chr>, Capital <int>, Code2 <chr>

In [26]:
df <- my_db %>% # es la conexión
  tbl("Country") %>% # extrae la tabla COuntry
    select(Code,Name) %>% # selecciona dos campos
      as.data.frame() # la vuelve data frame

In [27]:
head(df)

,Code,Name
,<chr>,<chr>
1,ABW,Aruba
2,AFG,Afghanistan
3,AGO,Angola
4,AIA,Anguilla
5,ALB,Albania
6,AND,Andorra


In [28]:
str(df)

'data.frame':	239 obs. of  2 variables:
 $ Code: chr  "ABW" "AFG" "AGO" "AIA" ...
 $ Name: chr  "Aruba" "Afghanistan" "Angola" "Anguilla" ...


# DBI

Otra forma de generar una búsqueda será con la librería DBI, utilizando el
comando dbSendQuery

In [32]:
library(DBI)
conn <- dbConnect(
  drv = RMySQL::MySQL(),
  dbname = "shinydemo",
  host = "shiny-demo.csa7qlmguqrf.us-east-1.rds.amazonaws.com",
  username = "guest",
  password = "guest"
  )

In [36]:
query <- "SELECT * FROM City LIMIT 7;"

In [37]:
rs <- dbSendQuery(conn, query)

In [38]:
dbFetch(rs)

ID,Name,CountryCode,District,Population
<dbl>,<chr>,<chr>,<chr>,<dbl>
1,Kabul,AFG,Kabol,1780000
2,Qandahar,AFG,Qandahar,237500
3,Herat,AFG,Herat,186800
4,Mazar-e-Sharif,AFG,Balkh,127800
5,Amsterdam,NLD,Noord-Holland,731200
6,Rotterdam,NLD,Zuid-Holland,593321
7,Haag,NLD,Zuid-Holland,440900


In [39]:
# Para finalizar nos desconectamos de la BDD

dbClearResult(rs)
dbDisconnect(conn)

[1] TRUE

[1] TRUE